# Tridiagonal Matrices
### Flora Hess (flora.hess@stud.uni-heidelberg.de)<br/>Leonardo K. Reiter (leonardo.reiter@stud.uni-heidelberg.de)<br/>Jason G. Jun (jun.jasongabriel@stud.uni-heidelberg.de)

In [1]:
import numpy as np
from scipy.sparse import diags
import matplotlib.pyplot as plt
%matplotlib inline

Consider the following tridiagonal $N \times N$ matrix equation

\begin{gather}
\begin{pmatrix}
b_1 & c_1 & 0 & 0 & \cdots & 0 & 0 & 0 & 0 \\
a_2 & b_2 & c_2 & 0 & \cdots & 0 & 0 & 0 & 0 \\
0 & a_3 & b_3 & c_3 & \cdots & 0 & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \vdots &   & \vdots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & 0 & \cdots & a_{N-2} & b_{N-2} & c_{N-2} & 0 \\
0 & 0 & 0 & 0 & \cdots & 0 & a_{N-1} & b_{N-1} & c_{N-1} \\
0 & 0 & 0 & 0 & \cdots & 0 & 0 & a_N & b_N 
\end{pmatrix}
\begin{pmatrix}
x_1  \\
x_2  \\
x_3  \\
\vdots \\
x_{N-2} \\
x_{N-1} \\
x_N  
\end{pmatrix}
=
\begin{pmatrix}
y_1  \\
y_2  \\
y_3  \\
\vdots \\
y_{N-2} \\
y_{N-1} \\
y_N  
\end{pmatrix}.
\end{gather}

### Exercise 1:
Derive the iterative expressions for Gaussian elimination, in a form that can be directly implemented as a numerical subroutine. Do not apply pivoting here, because in the special case of tridiagonal matrix equations pivoting is rarely necessary in practice. 

In [2]:
def forward_gauss(A, b):
    A, b = A.copy(), b.copy()
    for k in range(len(A)):
        for i in range(k+1, len(A)):
            A[i], b[i] = A[i] - A[k] / A[k][k] * A[i][k], b[i] - b[k] / A[k][k] * A[i][k]
    return A, b

### Exercise 2:
Derive the iterative expressions for backward substitution, also for implementation as a numerical subroutine. 

In [3]:
def backward_gauss(A, b):
    x = np.zeros(b.shape)
    for i in range(len(A)):
        x[-i-1] = (b[-i-1] - np.sum(A[-i-1][-i:] * x[-i:])) / A[-i-1][-i-1]
    return x

### Exercise 3:
Program a subroutine that, given the values $a_2 \cdots a_N$, $b_1 \cdots b_N$, $c_1 \cdots c_{N-1}$ and $y_1 \cdots y_N$, finds the solution vector given by $x_1 \cdots x_N$.

In [4]:
def find_solution(a, b, c, y):
    A = diags([a, b, c],[-1,0,1]).toarray().copy()
    return backward_gauss(*forward_gauss(A, y))

def text_solution(x):
    string = f"x   | Computed\n"
    for i in range(len(x)):
        n = chr(8320+(i+1)%10) if i < 9 else chr(8320+(i+1)//10)+chr(8320+(i+1)%10)
        string += f"x{n:<2} | {x[i]:>20.17f}\n"
    return string

def check_solution(a, b, c, x):
    A = diags([a, b, c],[-1,0,1]).toarray()
    return A@x

def text_deviation(y, y_sol):
    eps = np.finfo(np.float64).eps
    y_dev = abs( y_sol - y ) / eps
    string = "y   | Original            | Computed            | Deviation\n"
    for i in range(len(y)):
        n = chr(8320+(i+1)%10) if i < 9 else chr(8320+(i+1)//10)+chr(8320+(i+1)%10)
        string += f"y{n:<2} | {y[i]:>17.17} | {y_sol[i]:>17.17f} | {y_dev[i]:<.2f} eps\n"
    string += f"Machine Precision: 1 eps = {eps:<3.3e}\n"
    string += "All " if (y_dev<1).all() else "Not all "
    string += "deviations are within machine precision."
    return string

### Exercise 4:
Take $N = 10$, and set all $a$ values to $-1$, all $b$ values to $1.5$, all $c$ values to $-1$ and all $y$ values to $.1$. What is the solution for the $x_1 \cdots x_N$? 

In [5]:
N = 10
a, b, c, y = np.full(N-1, -1.), np.full(N, 1.5), np.full(N-1, -1.), np.full(N, .1)
x = find_solution(a, b, c, y)
print(text_solution(x))

x   | Computed
x₁  |  0.09565217391304352
x₂  |  0.04347826086956528
x₃  | -0.13043478260869562
x₄  | -0.33913043478260874
x₅  | -0.47826086956521757
x₆  | -0.47826086956521746
x₇  | -0.33913043478260863
x₈  | -0.13043478260869551
x₉  |  0.04347826086956538
x₁₀ |  0.09565217391304359



### Exercise 5:
Put your solution $x_1 \cdots x_N$ back into the original matrix equation above and find how much the result deviates from the original right-hand-side $y_1 \cdots y_N$. Is this satisfactory?

In [6]:
print(text_deviation( y, check_solution(a, b, c, x)))

y   | Original            | Computed            | Deviation
y₁  | 0.10000000000000001 | 0.10000000000000001 | 0.00 eps
y₂  | 0.10000000000000001 | 0.10000000000000002 | 0.06 eps
y₃  | 0.10000000000000001 | 0.10000000000000001 | 0.00 eps
y₄  | 0.10000000000000001 | 0.10000000000000009 | 0.38 eps
y₅  | 0.10000000000000001 | 0.09999999999999987 | 0.62 eps
y₆  | 0.10000000000000001 | 0.10000000000000009 | 0.38 eps
y₇  | 0.10000000000000001 | 0.09999999999999998 | 0.12 eps
y₈  | 0.10000000000000001 | 0.09999999999999995 | 0.25 eps
y₉  | 0.10000000000000001 | 0.09999999999999999 | 0.06 eps
y₁₀ | 0.10000000000000001 | 0.10000000000000001 | 0.00 eps
Machine Precision: 1 eps = 2.220e-16
All deviations are within machine precision.
